# Settling

The basic infrastructure for including settling dynamics in particle tracking simulations has been incorporated into PyLag's code base. Mathematically, in the vertical dimension, particle movement is modelled using the equation:

$$
\begin{equation}
    dZ\left(t\right) = \left(w + \dfrac{\partial K\left(z\right)}{\partial z}\right)dt + \left(2 K \left(z, t \right)\right)^{1/2} dW\left(t\right)
\end{equation}
$$

where $Z\left(t\right)$ is the position vector; $w$ is the vertical velocity, $K$ is the vertical eddy diffusivity, and $dW\left(t\right)$ is a Wiener noise term. Settling is incorporated by letting $w = w_{\mathrm{w}} + w_{\mathrm{s}}$, where $w_{\mathrm{w}}$ is the vertical velocity of the water and $w_{\mathrm{s}}$ is the particle's settling velocity.

PyLag includes an abstract base class called `SettlingVelocityCalculator`. All concrete settling velocity calculators subclass `SettlingVelocityCalculator`, and inherit its interface. If included in a simulation, they have the resonsibility for computing and setting particle settling velocities, which are included as a particle property. These are then used with the above equation, which is integrated numerically to compute changes in a particle's position through time.

Further information on the different types of settling velocity calculator included with PyLag are provided below. Information on adding new SettlingVelocityCalculators is also provided.

Typically, settling should be considered in combination with the type of condition employed at the bottom boundary, and the potential for resuspension once a particle has reached the sea bed. Further information on the types of bottom boundary condition implemented in PyLag can be found in the boundary conditions section of the documentation. Lastly, a tutorial example that includes settling dynamics can be found here.

## Settling velocity calculators

Currently, PyLag includes the following settling calculators:

1. **ConstantSettlingVelocityCalculator** - particles settle through the water column at a fixed rate.

2. **DelayedSettlingVelocityCalculator** - particles float at the surface for a specified period of time before starting to settle through the water column.

### 1. ConstantSettlingVelocityCalculator

Constant settling velocity calculators assign a fixed settling velocity to particles which doesn't change during the course of the simulation. The calculator is enabled through the following configuration options:

```python
[SETTLING]

settling_velocity_calculator = Constant
```

Particle settling velocities can be set in one of two ways. In the first, all particle's are assigned the same fixed settling velocity. In the second, a particle's settling velocity is drawn from a uniform random distribution with specified upper and lower limits. The selection is made using the following configuration options:

```python
[CONSTANT_SETTLING_VELOCITY_CALCUATOR]

initialisation_method = fixed_value

settling_velocity = 0.01

min_settling_velocity = 0.01

max_settling_velocity = 0.1
```

The configuration option `initialisation_method` can be assigned a value of `fixed_value` or `uniform_random`. All settling velocities have units of meters per second. If `initialisation_method` is set to `fixed_value`, all particles are given an identical settling velocity equal to `settling_velocity`. If it is set to `uniform_random`, particles are given a randomly assigned settling velocity between `min_settling_velocity` and `max_settling_velocity`.

### 2. DelayedSettlingVelocityCalculator

Delayed settling velocity calculators are designed to simulate the dynamics of particles that are initially positively buoyant, and sit at the ocean's surface, but then begin to sink after some specified period of time. A a fixed settling velocity is then assigned to particles. The calculator is enabled through the following configuration options:
```python
[SETTLING]

settling_velocity_calculator = Delayed
```

The parameters used with this particular calculator are as follows:

```python
[DELAYED_SETTLING_VELOCITY_CALCUATOR]

duration_of_surface_transport_phase_in_days = 6

settling_velocity = 0.01
```

The duration of time that a particle remains positivley buoyant for is set using the configuration option `duration_of_surface_transport_phase_in_days`. It's settling velocity once it begins to settle is set using the option `settling_velocity`, which has units of meters per second.


## Developing PyLag: Adding new settling velocity calculators

The process of adding new settling velocity calculators has three main steps:

1. Subclass `SettlingVelocityCalculator` in the module `settling.pyx`
2. Add the new settling velocity calculator to the factory method `get_settling_velocity_calculator(...)` in `settling.pyx`.
3. Add associated configuration options to the run configuration file using the same format outlined above.

To be effective, settling velocity calculators must compute and save a value for each particle's settling velocity. This becomes a property of the particle, which is saved as a diagnostic variable. It can be set using code of the form:

```python
particle.set_diagnostic_variable('particle_settling_velocity_in_meters_per_second, settling_velocity)
```

The settling velocity can be set once at model startup during a call to the method `init_particle_settling_velocity(...)`, or updated each time step through calls to `set_particle_settling_velocity(...)`. These calls are made automatically if a settling velocity calculator is being used, irrespective of its derived type, and no further modications of the code are required. The second method takes a `DataReader` object as a parameter, which gives the method the chance to access environmental variables that may influence the particle's settling velocity. Settling velocity calculators are also free to register and/or access additional particle parameters, variables and flags - such as a particle's mass, shape or size - and use these to compute settling rates. 